In [1]:
# ============================================================
# CONFIGURATION
# ============================================================

MODEL_PKL = "/kaggle/input/network-snapshot-0002097-0-100-logarithmic-2/pytorch/default/1/network-snapshot-0002097-0.100_logarithmic_noiseinject.pkl"

# Will be created after reference stats:
REF_PKL = "/kaggle/working/celeba64-ref-1k.pkl"

# Steps to evaluate
STEP_LIST = [8, 16, 24, 32]

# How many images for each FID run
NUM_IMAGES = 1000

# Output root directory
OUT_ROOT = "/kaggle/working/fid_vs_steps_logarithmic"
!mkdir -p {OUT_ROOT}


In [2]:
# ============================================================
# Clone EDM2 repo
# ============================================================

!git clone https://github.com/NVlabs/edm2.git /kaggle/working/edm2
%cd /kaggle/working/edm2
!pip install click tqdm psutil scipy pillow --quiet


Cloning into '/kaggle/working/edm2'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 60 (delta 13), reused 10 (delta 10), pack-reused 33 (from 1)
Receiving objects: 100% (60/60), 1.27 MiB | 9.51 MiB/s, done.
Resolving deltas: 100% (24/24), done.
/kaggle/working/edm2


In [3]:
# ============================================================
# Build 1k Reference Dataset (from 19,962 images)
# ============================================================
import os, shutil

SRC_DIR = "/kaggle/input/celeva-64x64-dataset/celeba64/test"
TEMP_REF_DIR = "/kaggle/working/ref_1k_imgs"

if not os.path.exists(TEMP_REF_DIR):
    print("🔧 Creating reference dataset with 1000 images...")
    os.makedirs(TEMP_REF_DIR, exist_ok=True)

    all_files = sorted(os.listdir(SRC_DIR))
    selected_files = all_files[:1000]   # deterministic selection

    for f in selected_files:
        shutil.copy(os.path.join(SRC_DIR, f), os.path.join(TEMP_REF_DIR, f))

    print("✅ Copied 1000 images into:", TEMP_REF_DIR)
else:
    print("📁 Reference 1k folder already exists:", TEMP_REF_DIR)


🔧 Creating reference dataset with 1000 images...
✅ Copied 1000 images into: /kaggle/working/ref_1k_imgs


In [4]:
# ============================================================
# Compute FID Reference Stats (1k images)
# ============================================================
import subprocess

if not os.path.exists(REF_PKL):
    print("🔄 Computing reference statistics (1k)...")

    cmd = f"""
        python calculate_metrics.py ref \
            --data={TEMP_REF_DIR} \
            --dest={REF_PKL} \
            --metrics=fid \
            --batch=64
    """

    print(subprocess.getoutput(cmd))

    print("✅ Saved reference stats:", REF_PKL)
else:
    print("📌 Using existing reference stats:", REF_PKL)


🔄 Computing reference statistics (1k)...
[rank0]:[W1116 02:45:32.405853462 ProcessGroupNCCL.cpp:4561] [PG ID 0 PG GUID 0 Rank 0]  using GPU 0 to perform barrier as devices used by this process are currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect. Specify device_ids in barrier() to force use of a particular device, or call init_process_group() with a device_id.
Loading images from /kaggle/working/ref_1k_imgs ...
Setting up InceptionV3Detector...
Calculating feature statistics...

100%|██████████| 16/16 [00:08<00:00,  1.91batch/s]
✅ Saved reference stats: /kaggle/working/celeba64-ref-1k.pkl


In [5]:
# ============================================================
# PATCH: Logarithmic Noise Sampler
# ============================================================
import torch, sys
sys.path.append("/kaggle/working/edm2")
import generate_images as gen

def logarithmic_t_steps(num_steps=32, sigma_min=0.002, sigma_max=80):
    """Logarithmic noise schedule: σ is log-spaced between sigma_max → sigma_min."""
    log_min = torch.log(torch.tensor(sigma_min, device='cuda'))
    log_max = torch.log(torch.tensor(sigma_max, device='cuda'))
    t = torch.linspace(0, 1, num_steps, device='cuda')
    sigmas = torch.exp(log_max - (log_max - log_min) * t)
    sigmas = torch.cat([sigmas, torch.zeros_like(sigmas[:1])])  # Append σ_N = 0
    return sigmas

def logarithmic_sampler(
    net, noise, labels=None, gnet=None,
    num_steps=32, sigma_min=0.002, sigma_max=80, guidance=1,
    S_churn=0, S_min=0, S_max=float('inf'), S_noise=1,
    dtype=torch.float32, randn_like=torch.randn_like,
):
    """EDM2 sampler modified for logarithmic σ schedule."""

    def denoise(x, t):
        Dx = net(x, t, labels).to(dtype)
        if guidance == 1:
            return Dx
        ref_Dx = gnet(x, t, labels).to(dtype)
        return ref_Dx.lerp(Dx, guidance)

    # ---- Logarithmic schedule ----
    t_steps = logarithmic_t_steps(num_steps, sigma_min, sigma_max)
    x_next = noise.to(dtype) * t_steps[0]

    # ---- Sampling Loop ----
    for i, (t_cur, t_next) in enumerate(zip(t_steps[:-1], t_steps[1:])):
        x_cur = x_next

        # Optional noise injection CHURN (unchanged)
        if S_churn > 0 and S_min <= t_cur <= S_max:
            gamma = min(S_churn / num_steps, torch.sqrt(torch.tensor(2.0)) - 1)
            t_hat = t_cur + gamma * t_cur
            x_hat = x_cur + (t_hat**2 - t_cur**2).sqrt() * S_noise * randn_like(x_cur)
        else:
            t_hat = t_cur
            x_hat = x_cur

        # Euler update
        d_cur = (x_hat - denoise(x_hat, t_hat)) / t_hat
        x_next = x_hat + (t_next - t_hat) * d_cur

        # 2nd order correction
        if i < num_steps - 1:
            d_prime = (x_next - denoise(x_next, t_next)) / t_next
            x_next = x_hat + (t_next - t_hat) * (0.5 * d_cur + 0.5 * d_prime)

    return x_next

# Inject into EDM2
gen.edm_sampler = logarithmic_sampler
print("✅ Logarithmic sampler patched successfully!")


✅ Logarithmic sampler patched successfully!


In [6]:
# ============================================================
# RUN FID VS STEPS (Logarithmic Noise Schedule)
# ============================================================
import subprocess
import re

results = []

for steps in STEP_LIST:
    print("\n==============================")
    print(f"🚀 Running steps={steps}")
    print("==============================")

    OUTDIR = f"{OUT_ROOT}/steps_{steps}"
    !mkdir -p {OUTDIR}

    # ---- IMAGE GENERATION ----
    print("🎨 Generating images...")
    !python generate_images.py \
        --net={MODEL_PKL} \
        --outdir={OUTDIR} \
        --seeds=0-{NUM_IMAGES-1} \
        --batch=32 \
        --steps={steps} \
        --sigma_min=0.002 --sigma_max=80 --rho=7

    # ---- COMPUTE FID ----
    print("📊 Computing FID...")
    cmd = f"""
        python calculate_metrics.py calc \
            --images={OUTDIR} \
            --ref={REF_PKL} \
            --metrics=fid \
            --num={NUM_IMAGES} \
            --batch=64
    """

    output = subprocess.getoutput(cmd)
    print(output)

    fid_match = re.search(r"fid\s*=\s*([\d.]+)", output)
    fid_value = float(fid_match.group(1)) if fid_match else None

    results.append((steps, fid_value))



🚀 Running steps=8
🎨 Generating images...
Loading main network from /kaggle/input/network-snapshot-0002097-0-100-logarithmic-2/pytorch/default/1/network-snapshot-0002097-0.100_logarithmic_noiseinject.pkl ...
Setting up StandardRGBEncoder...
[rank0]:[W1116 02:45:49.447567257 ProcessGroupNCCL.cpp:4561] [PG ID 0 PG GUID 0 Rank 0]  using GPU 0 to perform barrier as devices used by this process are currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect. Specify device_ids in barrier() to force use of a particular device, or call init_process_group() with a device_id.
Generating 1000 images...
  0%|                                                 | 0/32 [00:00<?, ?batch/s]/kaggle/working/edm2/generate_images.py:235: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  PIL.Image.fromarray(image, 'RGB').save(os.path.join(image_dir, f'{seed:06d}.png'))
100%|████████████████████████████████████████| 32/32 [02:03

In [7]:
# ============================================================
# SAVE RESULTS
# ============================================================
RESULT_TXT = f"{OUT_ROOT}/fid_vs_steps_logarithmic.txt"

with open(RESULT_TXT, "w") as f:
    for steps, fid in results:
        f.write(f"steps={steps}, FID={fid}\n")

print("\n🎉 DONE! Results saved at:", RESULT_TXT)
print(results)



🎉 DONE! Results saved at: /kaggle/working/fid_vs_steps_logarithmic/fid_vs_steps_logarithmic.txt
[(8, 46.2031), (16, 33.5049), (24, 32.4734), (32, 32.2019)]
